# Data Transformation

I have:
- data frame containing Landkreis IDs
- data frame containing Landkreis Locations
- data frame containing Weather Data

Goal:
- assign every Landkreis ONE weather station
- combine the three data frames into one

In [146]:
from pathlib import Path
import pandas as pd
import math
from tqdm import tqdm

In [2]:
path_base = Path.cwd()

# export path
path_export = Path.joinpath(path_base, "exports")
path_export.mkdir(parents=True, exist_ok=True)

In [3]:
# import the data from Notebook 01
df_temp = pd.read_pickle(Path.joinpath(path_export, "temp.pkl"))
df_temp_stations = pd.read_pickle(Path.joinpath(path_export, "temp_stations.pkl"))
df_prec = pd.read_pickle(Path.joinpath(path_export, "prec.pkl"))
df_prec_stations = pd.read_pickle(Path.joinpath(path_export, "prec_stations.pkl"))
df_sun = pd.read_pickle(Path.joinpath(path_export, "sun.pkl"))
df_sun_stations = pd.read_pickle(Path.joinpath(path_export, "sun_stations.pkl"))

## Integrate Geodata into Landkreise Frame

In [5]:
# load RKI Covid-19 data in order to build a Landkreis-ID lookup table
df_rki = pd.read_csv("https://www.arcgis.com/sharing/rest/content/items/f10774f1c63e40168479a1feb6c7ca74/data")
df_landkreise = df_rki.drop_duplicates('Landkreis')[['Landkreis', 'IdLandkreis', 'Bundesland', 'IdBundesland']]
df_landkreise

,Landkreis,IdLandkreis,Bundesland,IdBundesland
0,SK Flensburg,1001,Schleswig-Holstein,1
33,SK Kiel,1002,Schleswig-Holstein,1
284,SK Lübeck,1003,Schleswig-Holstein,1
437,SK Neumünster,1004,Schleswig-Holstein,1
500,LK Dithmarschen,1051,Schleswig-Holstein,1
...,...,...,...,...
126133,LK Saalfeld-Rudolstadt,16073,Thüringen,16
126194,LK Saale-Holzland-Kreis,16074,Thüringen,16
126246,LK Saale-Orla-Kreis,16075,Thüringen,16
126359,LK Greiz,16076,Thüringen,16


In [6]:
# load geographical data of the Landkreise in Germany
df_districts_geo = pd.read_csv("https://public.opendatasoft.com/explore/dataset/landkreise-in-germany/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B", ";")
df_districts_geo
# Our districtId is in column "Cca 2"

,Geo Point,Geo Shape,Id 0,ISO,Name 0,Id 1,Name 1,Id 2,Name 2,Hasc 2,Ccn 2,Cca 2,Type 2,Engtype 2,Nl Name 2,Varname 2
0,"47.9925229956,7.81807596197","{""type"": ""Polygon"", ""coordinates"": [[[7.790447...",86,DEU,Germany,1,Baden-Württemberg,12,Freiburg im Breisgau,DE.BW.FB,0,8311.0,Stadtkreis,District,NaN,NaN
1,"48.5964037974,10.527764168","{""type"": ""Polygon"", ""coordinates"": [[[10.61448...",86,DEU,Germany,2,Bayern,68,Dillingen an der Donau,DE.BY.DD,0,9773.0,Landkreis,District,NaN,NaN
2,"49.4362114486,11.0827553426","{""type"": ""MultiPolygon"", ""coordinates"": [[[[11...",86,DEU,Germany,2,Bayern,107,Nürnberg,DE.BY.NR,0,9564.0,Kreisfreie Stadt,District,NaN,NaN
3,"49.2159614099,11.5665579197","{""type"": ""Polygon"", ""coordinates"": [[[11.46063...",86,DEU,Germany,2,Bayern,110,Neumarkt in der Oberpfalz,DE.BY.NO,0,9373.0,Landkreis,District,NaN,NaN
4,"47.8443777181,12.1087247511","{""type"": ""Polygon"", ""coordinates"": [[[12.05431...",86,DEU,Germany,2,Bayern,122,Rosenheim,DE.BY.RH,0,9163.0,Kreisfreie Stadt,District,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,"49.416649982,8.36068108599","{""type"": ""Polygon"", ""coordinates"": [[[8.353994...",86,DEU,Germany,11,Rheinland-Pfalz,323,Rhein-Pfalz-Kreis,DE.RP.RZ,0,7338.0,Landkreis,District,NaN,NaN
399,"49.762938832,6.65505102152","{""type"": ""Polygon"", ""coordinates"": [[[6.734675...",86,DEU,Germany,11,Rheinland-Pfalz,328,Trier,DE.RP.TI,0,7211.0,Kreisfreie Stadt,District,NaN,NaN
400,"51.4256710773,11.865474038","{""type"": ""MultiPolygon"", ""coordinates"": [[[[11...",86,DEU,Germany,13,Sachsen-Anhalt,349,Saalekreis,DE.ST.SL,0,15088.0,Landkreis,District,NaN,NaN
401,"51.8201400674,12.7015882396","{""type"": ""Polygon"", ""coordinates"": [[[12.42280...",86,DEU,Germany,13,Sachsen-Anhalt,352,Wittenberg,DE.ST.WT,0,15091.0,Landkreis,District,NaN,NaN


In [149]:
df_lk = pd.merge(df_landkreise, df_districts_geo, left_on="IdLandkreis", right_on="Cca 2")[['Landkreis', 'Name 2', 'Type 2','IdLandkreis', 'Bundesland', 'Geo Point']]
df_lk = df_lk.rename(columns={'Name 2': 'Name kurz', 'Type 2': 'Typ'})
df_lk

,Landkreis,Name kurz,Typ,IdLandkreis,Bundesland,Geo Point
0,SK Flensburg,Flensburg,Kreisfreie Stadt,1001,Schleswig-Holstein,"54.7849933768,9.43852835486"
1,SK Kiel,Kiel,Kreisfreie Stadt,1002,Schleswig-Holstein,"54.3248406926,10.1322443646"
2,SK Lübeck,Lübeck,Kreisfreie Stadt,1003,Schleswig-Holstein,"53.8723167338,10.7272831058"
3,SK Neumünster,Neumünster,Kreisfreie Stadt,1004,Schleswig-Holstein,"54.0811244365,9.98448195474"
4,LK Dithmarschen,Dithmarschen,Kreis,1051,Schleswig-Holstein,"54.1329109614,9.10781447873"
...,...,...,...,...,...,...
394,LK Saalfeld-Rudolstadt,Saalfeld-Rudolstadt,Landkreis,16073,Thüringen,"50.637797959,11.3091162493"
395,LK Saale-Holzland-Kreis,Saale-Holzland-Kreis,Landkreis,16074,Thüringen,"50.904172137,11.7315307817"
396,LK Saale-Orla-Kreis,Saale-Orla-Kreis,Landkreis,16075,Thüringen,"50.5808480206,11.7105737336"
397,LK Greiz,Greiz,Landkreis,16076,Thüringen,"50.7484595538,12.0740705739"


In [150]:
# split up column "Geo Point" into two seperate numerical columns
df_lk['latitude'], df_lk['longitude'] = df_lk['Geo Point'].str.split(',', 1).str
df_lk[['latitude', 'longitude']] = df_lk[['latitude', 'longitude']].apply(pd.to_numeric)
df_lk.drop(columns=['Geo Point'], inplace=True)
df_lk

/Users/fynnwithake/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  


,Landkreis,Name kurz,Typ,IdLandkreis,Bundesland,latitude,longitude
0,SK Flensburg,Flensburg,Kreisfreie Stadt,1001,Schleswig-Holstein,54.784993,9.438528
1,SK Kiel,Kiel,Kreisfreie Stadt,1002,Schleswig-Holstein,54.324841,10.132244
2,SK Lübeck,Lübeck,Kreisfreie Stadt,1003,Schleswig-Holstein,53.872317,10.727283
3,SK Neumünster,Neumünster,Kreisfreie Stadt,1004,Schleswig-Holstein,54.081124,9.984482
4,LK Dithmarschen,Dithmarschen,Kreis,1051,Schleswig-Holstein,54.132911,9.107814
...,...,...,...,...,...,...,...
394,LK Saalfeld-Rudolstadt,Saalfeld-Rudolstadt,Landkreis,16073,Thüringen,50.637798,11.309116
395,LK Saale-Holzland-Kreis,Saale-Holzland-Kreis,Landkreis,16074,Thüringen,50.904172,11.731531
396,LK Saale-Orla-Kreis,Saale-Orla-Kreis,Landkreis,16075,Thüringen,50.580848,11.710574
397,LK Greiz,Greiz,Landkreis,16076,Thüringen,50.748460,12.074071


In [151]:
df_temp_stations

,station_id,start_date,end_date,altitude,latitude,longitude,name,state
0,3,1950-04-01,2011-03-31,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
1,44,2007-04-01,2020-05-02,44,52.9336,8.2370,Großenkneten,Niedersachsen
2,52,1976-01-01,1988-01-01,46,53.6623,10.1990,Ahrensburg-Wulfsdorf,Schleswig-Holstein
3,71,2009-12-01,2019-12-31,759,48.2156,8.9784,Albstadt-Badkap,Baden-Württemberg
4,73,2007-04-01,2020-05-02,340,48.6159,13.0506,Aldersbach-Kriestorf,Bayern
...,...,...,...,...,...,...,...,...
653,14138,2009-09-15,2015-12-31,73,52.1655,14.1224,Falkenberg (Grenzschichtmessfeld),Brandenburg
654,15000,2011-04-01,2020-05-02,231,50.7983,6.0244,Aachen-Orsbach,Nordrhein-Westfalen
655,15207,2013-11-01,2020-05-02,317,51.2835,9.3590,Schauenburg-Elgershausen,Hessen
656,15444,2014-09-01,2020-05-02,593,48.4418,9.9216,Ulm-Mähringen,Baden-Württemberg


### Landkreise that are not covered by this dataset
The RKI dataset gives data for 412 Landkreise, however, the dataset from _opendatasoft_ provides geospatial coordinates only for 399 of them.

The Landkreise for which no geospatial data exists will be neglected in the following. If we have a look at them, we see that its mostly the districts of Berlin that are special, so we treat Berlin as a whole in the future:

In [152]:
# some of the Landkreise are not covered by BOTH datasets, so they will be omitted
pd.concat([df_lk, df_landkreise]).drop_duplicates(['IdLandkreis'], keep=False)

,Landkreis,Name kurz,Typ,IdLandkreis,Bundesland,latitude,longitude,IdBundesland
6354,LK Göttingen,NaN,NaN,3159,Niedersachsen,NaN,NaN,3.0
110881,SK Berlin Mitte,NaN,NaN,11001,Berlin,NaN,NaN,11.0
111614,SK Berlin Friedrichshain-Kreuzberg,NaN,NaN,11002,Berlin,NaN,NaN,11.0
112032,SK Berlin Pankow,NaN,NaN,11003,Berlin,NaN,NaN,11.0
112577,SK Berlin Charlottenburg-Wilmersdorf,NaN,NaN,11004,Berlin,NaN,NaN,11.0
113192,SK Berlin Spandau,NaN,NaN,11005,Berlin,NaN,NaN,11.0
113414,SK Berlin Steglitz-Zehlendorf,NaN,NaN,11006,Berlin,NaN,NaN,11.0
113862,SK Berlin Tempelhof-Schöneberg,NaN,NaN,11007,Berlin,NaN,NaN,11.0
114420,SK Berlin Neukölln,NaN,NaN,11008,Berlin,NaN,NaN,11.0
115016,SK Berlin Treptow-Köpenick,NaN,NaN,11009,Berlin,NaN,NaN,11.0


### Add Göttingen
As _Geo Point_ I take the coordinates of the city of Göttingen.

In [153]:
df_lk = pd.concat([df_lk, df_landkreise[df_landkreise['IdLandkreis'] == 3159]])
df_lk.set_index('IdLandkreis', inplace=True)
df_lk

,Landkreis,Name kurz,Typ,Bundesland,latitude,longitude,IdBundesland
IdLandkreis,,,,,,,
1001,SK Flensburg,Flensburg,Kreisfreie Stadt,Schleswig-Holstein,54.784993,9.438528,NaN
1002,SK Kiel,Kiel,Kreisfreie Stadt,Schleswig-Holstein,54.324841,10.132244,NaN
1003,SK Lübeck,Lübeck,Kreisfreie Stadt,Schleswig-Holstein,53.872317,10.727283,NaN
1004,SK Neumünster,Neumünster,Kreisfreie Stadt,Schleswig-Holstein,54.081124,9.984482,NaN
1051,LK Dithmarschen,Dithmarschen,Kreis,Schleswig-Holstein,54.132911,9.107814,NaN
...,...,...,...,...,...,...,...
16074,LK Saale-Holzland-Kreis,Saale-Holzland-Kreis,Landkreis,Thüringen,50.904172,11.731531,NaN
16075,LK Saale-Orla-Kreis,Saale-Orla-Kreis,Landkreis,Thüringen,50.580848,11.710574,NaN
16076,LK Greiz,Greiz,Landkreis,Thüringen,50.748460,12.074071,NaN


In [154]:
df_lk.loc[3159, 'Name kurz'] = "Göttingen"
df_lk.loc[3159, 'Typ'] = "Landkreis"
df_lk.loc[3159, 'latitude'] = 51.540120
df_lk.loc[3159, 'longitude'] = 9.930627

df_lk.loc[3159]

Landkreis        LK Göttingen
Name kurz           Göttingen
Typ                 Landkreis
Bundesland      Niedersachsen
latitude              51.5401
longitude             9.93063
IdBundesland                3
Name: 3159, dtype: object

In [155]:
df_lk.reset_index(inplace=True)
df_lk

,IdLandkreis,Landkreis,Name kurz,Typ,Bundesland,latitude,longitude,IdBundesland
0,1001,SK Flensburg,Flensburg,Kreisfreie Stadt,Schleswig-Holstein,54.784993,9.438528,NaN
1,1002,SK Kiel,Kiel,Kreisfreie Stadt,Schleswig-Holstein,54.324841,10.132244,NaN
2,1003,SK Lübeck,Lübeck,Kreisfreie Stadt,Schleswig-Holstein,53.872317,10.727283,NaN
3,1004,SK Neumünster,Neumünster,Kreisfreie Stadt,Schleswig-Holstein,54.081124,9.984482,NaN
4,1051,LK Dithmarschen,Dithmarschen,Kreis,Schleswig-Holstein,54.132911,9.107814,NaN
...,...,...,...,...,...,...,...,...
395,16074,LK Saale-Holzland-Kreis,Saale-Holzland-Kreis,Landkreis,Thüringen,50.904172,11.731531,NaN
396,16075,LK Saale-Orla-Kreis,Saale-Orla-Kreis,Landkreis,Thüringen,50.580848,11.710574,NaN
397,16076,LK Greiz,Greiz,Landkreis,Thüringen,50.748460,12.074071,NaN
398,16077,LK Altenburger Land,Altenburger Land,Landkreis,Thüringen,50.956425,12.399131,NaN


## Strategy

- iterate over all Landkreise
- assign every Landkreis the weather station that is closest to it

In [181]:
def assign_weather_station_to_landkreis(df_stations, df_lk, df_weather):
    """Compares the center of each Landkreis with the location of each weather station
       and finds the one station that is closest to a particular Landkreis center.
    """
    
    # filter out stations that don't provide data in df_weather
    not_allowed = pd.concat([df_stations, df_weather]).drop_duplicates('station_id', keep=False)
    df_stations = df_stations[~df_stations['station_id'].isin(not_allowed['station_id'])]
    
    closest_station_dict = {}
    for lk_idx, lk_row in tqdm(df_lk.iterrows(), total=df_lk.shape[0]):
        idLandkreis = lk_row['IdLandkreis']
        for idx, row in df_stations.iterrows():
            
            # calculate distance between station and landkreis center
            lk_lat = lk_row['latitude']
            lk_lon = lk_row['longitude']

            station_lat = row['latitude']
            station_lon = row['longitude']

            a = station_lat - lk_lat
            b = station_lon - lk_lon
            distance = math.sqrt(a*a + b*b)

            if idLandkreis not in closest_station_dict.keys():
                closest_station_dict[idLandkreis] = {'station_id': row['station_id'], 'distance': distance}
            else:
                # check if current station is closer to landkreis
                if distance < closest_station_dict[idLandkreis]['distance']:
                    closest_station_dict[idLandkreis] = {'station_id': row['station_id'], 'distance': distance}
    df = pd.DataFrame.from_dict(closest_station_dict, orient='index').reset_index()
    df.rename(columns={'index': "IdLandkreis"}, inplace=True)
    return df

In [182]:
# these variables contain the assiciation of Landkreis to weather station
temp_lk_stations = assign_weather_station_to_landkreis(df_temp_stations, df_lk, df_temp)
prec_lk_stations = assign_weather_station_to_landkreis(df_prec_stations, df_lk, df_prec)
sun_lk_stations = assign_weather_station_to_landkreis(df_sun_stations, df_lk, df_sun)
sun_lk_stations




  0%|          | 0/400 [00:00<?, ?it/s]


  0%|          | 1/400 [00:00<00:54,  7.33it/s]


  0%|          | 2/400 [00:00<00:54,  7.35it/s]


  1%|          | 3/400 [00:00<00:51,  7.76it/s]


  1%|▏         | 5/400 [00:00<00:45,  8.67it/s]


  2%|▏         | 7/400 [00:00<00:41,  9.49it/s]


  2%|▏         | 8/400 [00:00<00:45,  8.55it/s]


  2%|▏         | 9/400 [00:00<00:45,  8.52it/s]


  2%|▎         | 10/400 [00:01<00:43,  8.90it/s]


  3%|▎         | 11/400 [00:01<00:43,  8.84it/s]


  3%|▎         | 12/400 [00:01<00:46,  8.43it/s]


  3%|▎         | 13/400 [00:01<00:46,  8.33it/s]


  4%|▎         | 14/400 [00:01<00:48,  8.00it/s]


  4%|▍         | 15/400 [00:01<00:48,  8.01it/s]


  4%|▍         | 16/400 [00:01<00:46,  8.27it/s]


  4%|▍         | 17/400 [00:01<00:48,  7.97it/s]


  4%|▍         | 18/400 [00:02<01:02,  6.13it/s]


  5%|▍         | 19/400 [00:02<01:05,  5.78it/s]


  5%|▌         | 20/400 [00:02<00:59,  6.36it/s]


  6%|▌         | 22/400 [00:02<00:50,  7.46i

 56%|█████▋    | 226/400 [00:23<00:16, 10.85it/s]


 57%|█████▋    | 228/400 [00:23<00:15, 11.35it/s]


 57%|█████▊    | 230/400 [00:23<00:16, 10.11it/s]


 58%|█████▊    | 232/400 [00:24<00:16, 10.49it/s]


 58%|█████▊    | 234/400 [00:24<00:15, 10.97it/s]


 59%|█████▉    | 236/400 [00:24<00:14, 10.96it/s]


 60%|█████▉    | 238/400 [00:24<00:14, 11.34it/s]


 60%|██████    | 240/400 [00:24<00:13, 11.84it/s]


 60%|██████    | 242/400 [00:24<00:13, 11.58it/s]


 61%|██████    | 244/400 [00:25<00:13, 11.91it/s]


 62%|██████▏   | 246/400 [00:25<00:12, 12.22it/s]


 62%|██████▏   | 248/400 [00:25<00:12, 12.24it/s]


 62%|██████▎   | 250/400 [00:25<00:11, 12.69it/s]


 63%|██████▎   | 252/400 [00:25<00:11, 12.89it/s]


 64%|██████▎   | 254/400 [00:25<00:11, 12.74it/s]


 64%|██████▍   | 256/400 [00:25<00:11, 12.71it/s]


 64%|██████▍   | 258/400 [00:26<00:11, 12.73it/s]


 65%|██████▌   | 260/400 [00:26<00:11, 12.52it/s]


 66%|██████▌   | 262/400 [00:26<00:11, 12.41it/s]


 66%|██████▌

  9%|▉         | 36/400 [00:05<00:49,  7.31it/s]


  9%|▉         | 37/400 [00:05<00:49,  7.34it/s]


 10%|▉         | 38/400 [00:05<00:50,  7.24it/s]


 10%|▉         | 39/400 [00:05<00:49,  7.29it/s]


 10%|█         | 40/400 [00:05<00:49,  7.31it/s]


 10%|█         | 41/400 [00:05<00:49,  7.32it/s]


 10%|█         | 42/400 [00:06<00:49,  7.20it/s]


 11%|█         | 43/400 [00:06<00:48,  7.29it/s]


 11%|█         | 44/400 [00:06<00:48,  7.34it/s]


 11%|█▏        | 45/400 [00:06<00:48,  7.34it/s]


 12%|█▏        | 46/400 [00:06<00:51,  6.92it/s]


 12%|█▏        | 47/400 [00:06<01:04,  5.47it/s]


 12%|█▏        | 48/400 [00:06<01:00,  5.79it/s]


 12%|█▏        | 49/400 [00:07<00:57,  6.15it/s]


 12%|█▎        | 50/400 [00:07<01:04,  5.41it/s]


 13%|█▎        | 51/400 [00:07<01:03,  5.52it/s]


 13%|█▎        | 52/400 [00:07<00:58,  5.93it/s]


 13%|█▎        | 53/400 [00:07<00:54,  6.31it/s]


 14%|█▎        | 54/400 [00:07<00:55,  6.26it/s]


 14%|█▍        | 55/400 [00:08<

 43%|████▎     | 171/400 [00:26<00:47,  4.77it/s]


 43%|████▎     | 172/400 [00:26<00:44,  5.14it/s]


 43%|████▎     | 173/400 [00:27<00:42,  5.34it/s]


 44%|████▎     | 174/400 [00:27<00:40,  5.60it/s]


 44%|████▍     | 175/400 [00:27<00:38,  5.82it/s]


 44%|████▍     | 176/400 [00:27<00:38,  5.81it/s]


 44%|████▍     | 177/400 [00:27<00:46,  4.83it/s]


 44%|████▍     | 178/400 [00:28<00:47,  4.72it/s]


 45%|████▍     | 179/400 [00:28<00:43,  5.11it/s]


 45%|████▌     | 180/400 [00:28<00:48,  4.50it/s]


 45%|████▌     | 181/400 [00:28<00:48,  4.49it/s]


 46%|████▌     | 182/400 [00:28<00:44,  4.85it/s]


 46%|████▌     | 183/400 [00:29<00:42,  5.11it/s]


 46%|████▌     | 184/400 [00:29<00:40,  5.36it/s]


 46%|████▋     | 185/400 [00:29<00:38,  5.52it/s]


 46%|████▋     | 186/400 [00:29<00:39,  5.45it/s]


 47%|████▋     | 187/400 [00:29<00:39,  5.45it/s]


 47%|████▋     | 188/400 [00:29<00:37,  5.61it/s]


 47%|████▋     | 189/400 [00:30<00:38,  5.52it/s]


 48%|████▊  

 76%|███████▋  | 305/400 [00:49<00:13,  7.27it/s]


 76%|███████▋  | 306/400 [00:49<00:12,  7.26it/s]


 77%|███████▋  | 307/400 [00:50<00:12,  7.17it/s]


 77%|███████▋  | 308/400 [00:50<00:12,  7.25it/s]


 77%|███████▋  | 309/400 [00:50<00:12,  7.28it/s]


 78%|███████▊  | 310/400 [00:50<00:12,  7.19it/s]


 78%|███████▊  | 311/400 [00:50<00:12,  7.27it/s]


 78%|███████▊  | 312/400 [00:50<00:12,  7.31it/s]


 78%|███████▊  | 313/400 [00:50<00:11,  7.31it/s]


 78%|███████▊  | 314/400 [00:51<00:12,  7.13it/s]


 79%|███████▉  | 315/400 [00:51<00:11,  7.18it/s]


 79%|███████▉  | 316/400 [00:51<00:11,  7.22it/s]


 79%|███████▉  | 317/400 [00:51<00:11,  7.13it/s]


 80%|███████▉  | 318/400 [00:51<00:11,  7.21it/s]


 80%|███████▉  | 319/400 [00:51<00:11,  7.30it/s]


 80%|████████  | 320/400 [00:51<00:10,  7.32it/s]


 80%|████████  | 321/400 [00:51<00:11,  7.17it/s]


 80%|████████  | 322/400 [00:52<00:10,  7.28it/s]


 81%|████████  | 323/400 [00:52<00:10,  7.33it/s]


 81%|███████

 20%|██        | 80/400 [00:05<00:18, 16.95it/s]


 20%|██        | 82/400 [00:05<00:18, 16.77it/s]


 21%|██        | 84/400 [00:05<00:18, 17.23it/s]


 22%|██▏       | 86/400 [00:05<00:17, 17.58it/s]


 22%|██▏       | 88/400 [00:06<00:17, 18.18it/s]


 22%|██▎       | 90/400 [00:06<00:16, 18.35it/s]


 23%|██▎       | 92/400 [00:06<00:16, 18.13it/s]


 24%|██▎       | 94/400 [00:06<00:18, 16.53it/s]


 24%|██▍       | 96/400 [00:06<00:18, 16.80it/s]


 24%|██▍       | 98/400 [00:06<00:25, 11.93it/s]


 25%|██▌       | 100/400 [00:06<00:24, 12.46it/s]


 26%|██▌       | 102/400 [00:07<00:21, 13.61it/s]


 26%|██▌       | 104/400 [00:07<00:19, 15.01it/s]


 26%|██▋       | 106/400 [00:07<00:18, 16.15it/s]


 27%|██▋       | 108/400 [00:07<00:17, 16.36it/s]


 28%|██▊       | 110/400 [00:07<00:16, 17.18it/s]


 28%|██▊       | 112/400 [00:07<00:27, 10.50it/s]


 28%|██▊       | 114/400 [00:07<00:24, 11.50it/s]


 29%|██▉       | 116/400 [00:08<00:22, 12.90it/s]


 30%|██▉       | 118/4

100%|██████████| 400/400 [00:24<00:00, 16.16it/s]


,IdLandkreis,station_id,distance
0,1001,4466,0.280073
1,1002,6163,0.271422
2,1003,3086,0.075366
3,1004,6105,0.298250
4,1051,1200,0.116370
...,...,...,...
395,16074,7420,0.201257
396,16075,4464,0.094418
397,16076,1612,0.143711
398,16077,1612,0.280479


In [185]:
prec_lk_stations

,IdLandkreis,station_id,distance
0,1001,1130,0.173220
1,1002,2564,0.053728
2,1003,4602,0.072251
3,1004,7427,0.085809
4,1051,1200,0.116370
...,...,...,...
395,16074,550,0.066969
396,16075,2992,0.073163
397,16076,7419,0.087473
398,16077,4997,0.060851


## Plot Landkreis-Weather Station Assignment

In [186]:
temp_stations = temp_lk_stations.merge(df_lk, on='IdLandkreis').merge(df_temp_stations, on='station_id').drop(columns=['Name kurz', 'Typ', 'Bundesland', 'start_date', 'end_date', 'state'])
temp_stations.rename(columns={'latitude_x': 'lk_latitude', 'longitude_x': 'lk_longitude', 'latitude_y': 'station_latitude', 'longitude_y': 'station_longitude'}, inplace=True)

prec_stations = prec_lk_stations.merge(df_lk, on='IdLandkreis').merge(df_prec_stations, on='station_id').drop(columns=['Name kurz', 'Typ', 'Bundesland', 'start_date', 'end_date', 'state'])
prec_stations.rename(columns={'latitude_x': 'lk_latitude', 'longitude_x': 'lk_longitude', 'latitude_y': 'station_latitude', 'longitude_y': 'station_longitude'}, inplace=True)

sun_stations = sun_lk_stations.merge(df_lk, on='IdLandkreis').merge(df_sun_stations, on='station_id').drop(columns=['Name kurz', 'Typ', 'Bundesland', 'start_date', 'end_date', 'state'])
sun_stations.rename(columns={'latitude_x': 'lk_latitude', 'longitude_x': 'lk_longitude', 'latitude_y': 'station_latitude', 'longitude_y': 'station_longitude'}, inplace=True)


prec_stations

,IdLandkreis,station_id,distance,Landkreis,lk_latitude,lk_longitude,IdBundesland,altitude,station_latitude,station_longitude,name
0,1001,1130,0.173220,SK Flensburg,54.784993,9.438528,NaN,17,54.6282,9.3649,Eggebek
1,1002,2564,0.053728,SK Kiel,54.324841,10.132244,NaN,28,54.3776,10.1424,Kiel-Holtenau
2,1003,4602,0.072251,SK Lübeck,53.872317,10.727283,NaN,26,53.9385,10.6983,"Schwartau,Bad -Groß Parin"
3,1004,7427,0.085809,SK Neumünster,54.081124,9.984482,NaN,17,54.0188,9.9255,Padenstedt (Pony-Park)
4,1051,1200,0.116370,LK Dithmarschen,54.132911,9.107814,NaN,3,54.0691,9.0105,Elpersbüttel
...,...,...,...,...,...,...,...,...,...,...,...
395,16074,550,0.066969,LK Saale-Holzland-Kreis,50.904172,11.731531,NaN,344,50.9041,11.7985,Bobeck
396,16075,2992,0.073163,LK Saale-Orla-Kreis,50.580848,11.710574,NaN,495,50.5276,11.6604,Remptendorf
397,16076,7419,0.087473,LK Greiz,50.748460,12.074071,NaN,389,50.6610,12.0756,Langenwetzendorf-Göttendorf
398,16077,4997,0.060851,LK Altenburger Land,50.956425,12.399131,NaN,196,50.9771,12.3419,Starkenberg-Tegkwitz


## Build Final Dataframe
Eventually, we want to have weather parameters for each day for each Landkreis. So far, the measurements are on an hourly resolution. I take the daily mean of the temperatures, and the sum of the precipitatino and sunshine hour data per day. 

Finally, all data is merged into a single dataframe that holds the temperature, precipitation and sunshine measurements for a particular day in a given Landkreis in one row.

In [188]:
#df_temp.groupby(['station_id', pd.Grouper(key='date', freq='D')]).mean().reset_index().merge(temp_lk_stations, on='station_id').merge(df_lk, on="IdLandkreis")
temp = df_temp.groupby(['station_id', pd.Grouper(key='date', freq='D')]).mean().reset_index()
prec = df_prec.groupby(['station_id', pd.Grouper(key='date', freq='D')]).sum().reset_index()
sun = df_sun.groupby(['station_id', pd.Grouper(key='date', freq='D')]).sum().reset_index()

In [190]:
temp

,station_id,date,quality,temperature,humidity
0,44,2020-01-01,3,-1.045833,98.625000
1,44,2020-01-02,3,-0.045833,97.458333
2,44,2020-01-03,3,6.495833,92.666667
3,44,2020-01-04,3,4.762500,88.916667
4,44,2020-01-05,3,4.162500,92.625000
...,...,...,...,...,...
60634,15555,2020-04-28,1,12.466667,70.250000
60635,15555,2020-04-29,1,7.837500,84.791667
60636,15555,2020-04-30,1,8.462500,77.375000
60637,15555,2020-05-01,1,7.833333,81.125000


In [191]:
temp_lk_stations

,IdLandkreis,station_id,distance
0,1001,1666,0.079469
1,1002,2564,0.053728
2,1003,3086,0.075366
3,1004,7427,0.085809
4,1051,1200,0.116370
...,...,...,...
395,16074,2444,0.149998
396,16075,4464,0.094418
397,16076,7419,0.087473
398,16077,4997,0.060851


In [193]:
temp_final = temp.merge(temp_lk_stations, on="station_id").merge(df_lk, on="IdLandkreis").merge(df_temp_stations, on="station_id")
temp_final.drop(columns=['quality', 'station_id', 'Name kurz', 'start_date', 'end_date', 'altitude', 'state', 'Typ', 'Bundesland'], inplace=True)
temp_final.rename(columns={'latitude_x': 'lk_latitude', 'longitude_x': 'lk_longitude', 'latitude_y': 'station_latitude', 'longitude_y': 'station_longitude', 'name': 'station_name'}, inplace=True)
temp_final = temp_final[['date', 'temperature', 'humidity', 'IdLandkreis', 'Landkreis', 'lk_latitude', 'lk_longitude', 'station_name', 'station_latitude', 'station_longitude']]
temp_final

,date,temperature,humidity,IdLandkreis,Landkreis,lk_latitude,lk_longitude,station_name,station_latitude,station_longitude
0,2020-01-01,-1.045833,98.625000,3403,SK Oldenburg,53.144578,8.224359,Großenkneten,52.9336,8.2370
1,2020-01-02,-0.045833,97.458333,3403,SK Oldenburg,53.144578,8.224359,Großenkneten,52.9336,8.2370
2,2020-01-03,6.495833,92.666667,3403,SK Oldenburg,53.144578,8.224359,Großenkneten,52.9336,8.2370
3,2020-01-04,4.762500,88.916667,3403,SK Oldenburg,53.144578,8.224359,Großenkneten,52.9336,8.2370
4,2020-01-05,4.162500,92.625000,3403,SK Oldenburg,53.144578,8.224359,Großenkneten,52.9336,8.2370
...,...,...,...,...,...,...,...,...,...,...
49124,2020-04-28,12.466667,70.250000,9778,LK Unterallgäu,48.039813,10.389233,Kaufbeuren-Oberbeuren,47.8761,10.5848
49125,2020-04-29,7.837500,84.791667,9778,LK Unterallgäu,48.039813,10.389233,Kaufbeuren-Oberbeuren,47.8761,10.5848
49126,2020-04-30,8.462500,77.375000,9778,LK Unterallgäu,48.039813,10.389233,Kaufbeuren-Oberbeuren,47.8761,10.5848
49127,2020-05-01,7.833333,81.125000,9778,LK Unterallgäu,48.039813,10.389233,Kaufbeuren-Oberbeuren,47.8761,10.5848


In [195]:
prec_final = prec.merge(prec_lk_stations, on="station_id").merge(df_lk, on="IdLandkreis").merge(df_prec_stations, on="station_id")
prec_final.drop(columns=['R1_IND', 'quality', 'station_id', 'WRTR', 'Name kurz', 'start_date', 'end_date', 'altitude', 'state', 'Typ', 'Bundesland'], inplace=True)
prec_final.rename(columns={'R1': 'precipitation', 'latitude_x': 'lk_latitude', 'longitude_x': 'lk_longitude', 'latitude_y': 'station_latitude', 'longitude_y': 'station_longitude', 'name': 'station_name'}, inplace=True)
prec_final = prec_final[['date', 'precipitation', 'IdLandkreis', 'Landkreis', 'lk_latitude', 'lk_longitude', 'station_name', 'station_latitude', 'station_longitude']]
prec_final

,date,precipitation,IdLandkreis,Landkreis,lk_latitude,lk_longitude,station_name,station_latitude,station_longitude
0,2020-01-01,0.0,6535,LK Vogelsbergkreis,50.638119,9.271380,Alsfeld-Eifa,50.7446,9.3450
1,2020-01-02,0.0,6535,LK Vogelsbergkreis,50.638119,9.271380,Alsfeld-Eifa,50.7446,9.3450
2,2020-01-03,0.7,6535,LK Vogelsbergkreis,50.638119,9.271380,Alsfeld-Eifa,50.7446,9.3450
3,2020-01-04,2.7,6535,LK Vogelsbergkreis,50.638119,9.271380,Alsfeld-Eifa,50.7446,9.3450
4,2020-01-05,0.4,6535,LK Vogelsbergkreis,50.638119,9.271380,Alsfeld-Eifa,50.7446,9.3450
...,...,...,...,...,...,...,...,...,...
48952,2020-04-28,15.2,9777,LK Ostallgäu,47.769959,10.639732,Kaufbeuren-Oberbeuren,47.8761,10.5848
48953,2020-04-29,25.9,9777,LK Ostallgäu,47.769959,10.639732,Kaufbeuren-Oberbeuren,47.8761,10.5848
48954,2020-04-30,4.1,9777,LK Ostallgäu,47.769959,10.639732,Kaufbeuren-Oberbeuren,47.8761,10.5848
48955,2020-05-01,2.2,9777,LK Ostallgäu,47.769959,10.639732,Kaufbeuren-Oberbeuren,47.8761,10.5848


In [196]:
sun_final = sun.merge(sun_lk_stations, on="station_id").merge(df_lk, on="IdLandkreis").merge(df_sun_stations, on="station_id")
sun_final.drop(columns=['quality', 'station_id', 'Name kurz', 'start_date', 'end_date', 'altitude', 'state', 'Typ', 'Bundesland'], inplace=True)
sun_final.rename(columns={'SD_SO': 'sunshine', 'latitude_x': 'lk_latitude', 'longitude_x': 'lk_longitude', 'latitude_y': 'station_latitude', 'longitude_y': 'station_longitude', 'name': 'station_name'}, inplace=True)
sun_final = sun_final[['date', 'sunshine', 'IdLandkreis', 'Landkreis', 'lk_latitude', 'lk_longitude', 'station_name', 'station_latitude', 'station_longitude']]
sun_final

,date,sunshine,IdLandkreis,Landkreis,lk_latitude,lk_longitude,station_name,station_latitude,station_longitude
0,2020-01-01,0.0,3403,SK Oldenburg,53.144578,8.224359,Großenkneten,52.9336,8.2370
1,2020-01-02,0.0,3403,SK Oldenburg,53.144578,8.224359,Großenkneten,52.9336,8.2370
2,2020-01-03,0.0,3403,SK Oldenburg,53.144578,8.224359,Großenkneten,52.9336,8.2370
3,2020-01-04,0.0,3403,SK Oldenburg,53.144578,8.224359,Großenkneten,52.9336,8.2370
4,2020-01-05,0.0,3403,SK Oldenburg,53.144578,8.224359,Großenkneten,52.9336,8.2370
...,...,...,...,...,...,...,...,...,...
48232,2020-04-28,235.0,9778,LK Unterallgäu,48.039813,10.389233,Kaufbeuren-Oberbeuren,47.8761,10.5848
48233,2020-04-29,243.0,9778,LK Unterallgäu,48.039813,10.389233,Kaufbeuren-Oberbeuren,47.8761,10.5848
48234,2020-04-30,60.0,9778,LK Unterallgäu,48.039813,10.389233,Kaufbeuren-Oberbeuren,47.8761,10.5848
48235,2020-05-01,191.0,9778,LK Unterallgäu,48.039813,10.389233,Kaufbeuren-Oberbeuren,47.8761,10.5848


In [198]:
sun_final.groupby('IdLandkreis').count()

,date,sunshine,Landkreis,lk_latitude,lk_longitude,station_name,station_latitude,station_longitude
IdLandkreis,,,,,,,,
1001,123,123,123,123,123,123,123,123
1002,123,123,123,123,123,123,123,123
1003,123,123,123,123,123,123,123,123
1004,123,123,123,123,123,123,123,123
1051,123,123,123,123,123,123,123,123
...,...,...,...,...,...,...,...,...
16073,123,123,123,123,123,123,123,123
16074,123,123,123,123,123,123,123,123
16075,121,121,121,121,121,121,121,121


## Export

In [199]:
temp_final.to_pickle(Path.joinpath(path_export, "temp_final.pkl"))
prec_final.to_pickle(Path.joinpath(path_export, "prec_final.pkl"))
sun_final.to_pickle(Path.joinpath(path_export, "sun_final.pkl"))

df_lk.to_pickle(Path.joinpath(path_export, "landkreise.pkl"))

temp_stations.to_pickle(Path.joinpath(path_export, "temp_stations_assigned.pkl"))
prec_stations.to_pickle(Path.joinpath(path_export, "prec_stations_assigned.pkl"))
sun_stations.to_pickle(Path.joinpath(path_export, "sun_stations_assigned.pkl"))


In [200]:
prec_stations

,IdLandkreis,station_id,distance,Landkreis,lk_latitude,lk_longitude,IdBundesland,altitude,station_latitude,station_longitude,name
0,1001,1130,0.173220,SK Flensburg,54.784993,9.438528,NaN,17,54.6282,9.3649,Eggebek
1,1002,2564,0.053728,SK Kiel,54.324841,10.132244,NaN,28,54.3776,10.1424,Kiel-Holtenau
2,1003,4602,0.072251,SK Lübeck,53.872317,10.727283,NaN,26,53.9385,10.6983,"Schwartau,Bad -Groß Parin"
3,1004,7427,0.085809,SK Neumünster,54.081124,9.984482,NaN,17,54.0188,9.9255,Padenstedt (Pony-Park)
4,1051,1200,0.116370,LK Dithmarschen,54.132911,9.107814,NaN,3,54.0691,9.0105,Elpersbüttel
...,...,...,...,...,...,...,...,...,...,...,...
395,16074,550,0.066969,LK Saale-Holzland-Kreis,50.904172,11.731531,NaN,344,50.9041,11.7985,Bobeck
396,16075,2992,0.073163,LK Saale-Orla-Kreis,50.580848,11.710574,NaN,495,50.5276,11.6604,Remptendorf
397,16076,7419,0.087473,LK Greiz,50.748460,12.074071,NaN,389,50.6610,12.0756,Langenwetzendorf-Göttendorf
398,16077,4997,0.060851,LK Altenburger Land,50.956425,12.399131,NaN,196,50.9771,12.3419,Starkenberg-Tegkwitz
